In [1]:
from tqdm.notebook import tqdm
import igl
import meshio
import numpy as np
import os
import pandas as pd
import pathlib
import subprocess
import json

In [2]:
# tols = 10**np.arange(-2, 4, dtype=float)[::-1]

# with open("../scripts/cantilever/cantilever.json", "r") as f:
#     nref = json.load(f)["geometry"][0]["n_refs"]
    
# for tol in tqdm(tols):
#     exp = int(np.log10(tol))
#     script = pathlib.Path(f"../scripts/cantilever/convergence/tol=1e{exp}.json").resolve()
#     subprocess.run([
#         "/Users/zachary/Development/grad-research/polyfem/polyfem/build/release/PolyFEM_bin",
#         "-j", str(script),
#         "-o", f"/Users/zachary/Desktop/cantilever-convergence/nref={nref}/tol=1e{exp}"
#     ], stdout=open(os.devnull, "wb"), stderr=subprocess.STDOUT)

In [3]:
results_root = pathlib.Path("/Users/zachary/Desktop/cantilever-convergence/")

In [4]:
def foo(mesh_path, results, tip=[48, 60, 0]):
    if not mesh_path.exists():
        return
    mesh = meshio.read(mesh_path)
    pi = np.argwhere(np.linalg.norm(mesh.points - tip, axis=1) < 1e-12)[0]
    # results["Tip y-displacement (in mm)"] = float(mesh.point_data["solution"][pi, 1])
    results["Tip displacement (in mm)"] = np.linalg.norm(mesh.point_data["solution"][pi])
    V, _, _, _ = igl.remove_duplicate_vertices(mesh.points.astype(float), mesh.cells[0].data.astype(int), 1e-12)
    results["Number of degrees of freedom"] = V.shape[0] * 2

In [5]:
n_edges = (2**np.arange(9)).tolist()
columns=["Tip displacement (in mm)", "Number of degrees of freedom"]
index_name = "Elements per edge"

Q1_results = pd.DataFrame(index=n_edges, columns=columns)
Q1_results.index.name = index_name

for n in n_edges:
    foo(results_root / "Q1" / f"edges={n}" / "step_10.vtu", Q1_results.loc[n])

print(Q1_results)

                  Tip displacement (in mm) Number of degrees of freedom
Elements per edge                                                      
1                                      NaN                          NaN
2                                      NaN                          NaN
4                                      NaN                          NaN
8                                      NaN                          NaN
16                                     NaN                          NaN
32                                     NaN                          NaN
64                                     NaN                          NaN
128                                    NaN                          NaN
256                                    NaN                          NaN


In [6]:
Q2_results = pd.DataFrame(index=n_edges, columns=columns)
Q2_results.index.name = index_name

for n in n_edges:
    foo(results_root / "Q2" / f"edges={n}" / "step_10.vtu", Q2_results.loc[n])
    Q2_results.loc[n]["Number of degrees of freedom"] = Q1_results.loc[n]["Number of degrees of freedom"] + (2 * (n+1)**2)

print(Q2_results)

                  Tip displacement (in mm) Number of degrees of freedom
Elements per edge                                                      
1                                      NaN                          NaN
2                                      NaN                          NaN
4                                      NaN                          NaN
8                                      NaN                          NaN
16                                     NaN                          NaN
32                                     NaN                          NaN
64                                     NaN                          NaN
128                                    NaN                          NaN
256                                    NaN                          NaN


In [7]:
tols = 10**np.arange(-2, 4, dtype=float)[::-1]
results = pd.DataFrame(index=tols, columns=columns)
results.index.name = "Split tolerance"

our_results = []
for nref in range(1, 4):
    tmp = results.copy()
    for tol in tols:
        foo(results_root / f"nref={nref}" / f"tol=1e{int(np.log10(tol))}" / "step_30.vtu", tmp.loc[tol])
    our_results.append(tmp.copy())
    
tmp = results.copy()
for tol in tols:
    foo(results_root / "with-swap" / f"tol=1e{int(np.log10(tol))}" / "step_30.vtu", tmp.loc[tol])
our_results.append(tmp.copy())

print(our_results)

[                Tip displacement (in mm) Number of degrees of freedom
Split tolerance                                                      
1000.00                              NaN                          NaN
100.00                               NaN                          NaN
10.00                                NaN                          NaN
1.00                                 NaN                          NaN
0.10                                 NaN                          NaN
0.01                                 NaN                          NaN,                 Tip displacement (in mm) Number of degrees of freedom
Split tolerance                                                      
1000.00                              NaN                          NaN
100.00                               NaN                          NaN
10.00                                NaN                          NaN
1.00                                 NaN                          NaN
0.10              

In [8]:
import plotly.graph_objects as go
from paper_style import *

line_width = 3
common=dict(line_width=line_width, marker_size=3*line_width, mode="lines+markers+text")

fig = go.Figure(data=[
    go.Scatter(
        x=Q1_results[1:-1]["Number of degrees of freedom"], y=Q1_results[1:-1]["Tip displacement (in mm)"],  line_color=matlab_colors[0], name="Uniform refinement",
        # line=dict(dash='dash'),
        **common),
    go.Scatter(x=our_results[0]["Number of degrees of freedom"], y=our_results[0]["Tip displacement (in mm)"], line_color=matlab_colors[1], name=f"Ours (1)", **common),
    go.Scatter(x=our_results[1]["Number of degrees of freedom"], y=our_results[1]["Tip displacement (in mm)"], line_color=matlab_colors[2], name=f"Ours (2)", **common),
    go.Scatter(x=our_results[2]["Number of degrees of freedom"], y=our_results[2]["Tip displacement (in mm)"], line_color=matlab_colors[3], name=f"Ours (3)", **common),
    go.Scatter(x=our_results[-1]["Number of degrees of freedom"], y=our_results[-1]["Tip displacement (in mm)"], line_color=matlab_colors[4], name="Ours (swap+smooth)", **common),
    # go.Scatter(x=Q2_results[1:]["Number of degrees of freedom"], y=Q2_results[1:]["Tip displacement (in mm)"], name="Q2", line_color=matlab_colors[3], **common),
])

for tol in tols:
    text = r"${}{:g}~\text{{{}J}}$".format(
        r'\delta_s = ' if tol == 1e3 else '',
        tol / (1000 if tol >= 100  else (1/1000 if tol <= 1e-2 else 1)), 
        "μ" if tol >= 100 else ("p" if tol <= 1e-2 else "n")
    )
    fig.add_annotation(
        x=np.log10(our_results[-1].loc[tol]["Number of degrees of freedom"]),
        y=our_results[-1].loc[tol]["Tip displacement (in mm)"],
        text=text,
        font=dict(
            family="Linux Biolinum O",
            size=font_size*0.8,
            color=matlab_colors[4]
        ),
        align="center",
        showarrow=False,
        arrowhead=3,
        arrowcolor=matlab_colors[4],
        arrowwidth=2,
        yshift=20 if tol >= 100 else -20,
        xshift=-30 if tol > 100 else 0,
        ay=40
    )

fig.update_layout(
    # title_text="Cantilever Convergence",
    xaxis_title="Degrees of Freedom",
    yaxis_title="Tip Displacement (mm)",
    xaxis_type="log",
    # xaxis_dtick=1,
    **paper_style(aspect_ratio=1/2),
)

fig.show()
fig.write_image("cantilever.pdf")